In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A30


In [3]:
# Import dataset from saved PyTorch dataset
data = torch.load("/dfs6/pub/seminl1/graph500/graph500_scale23_ef64.pt")

# Store the dataset to GPU
#data = data.pin_memory()
#data = data.to('cuda:0', non_blocking=True)

In [4]:
# Transform dense edge index to sparse adjacency matrix
import torch_geometric.transforms as T

data = T.ToSparseTensor()(data)

In [5]:
# Check whether the dataset is stored on the GPU or not
print(f'Graph is stored on the GPU: {data.is_cuda}')

Graph is stored on the GPU: False


In [6]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[8388608, 64], y=[8388608], train_mask=[8388608], adj_t=[8388608, 8388608, nnz=517557419])


In [7]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([8388608, 64])
tensor([[-1.6241, -1.1809,  1.4980,  ...,  1.0852, -1.1694,  0.5091],
        [-1.3206, -1.1632, -2.0369,  ..., -2.1283, -1.8054, -1.2949],
        [-1.2896,  1.1596,  1.6661,  ..., -0.4917,  1.7394,  2.0564],
        ...,
        [ 1.0796,  1.1860, -0.0762,  ..., -1.8736,  1.2661,  0.6824],
        [-1.4856,  2.1285, -1.8341,  ..., -1.8353,  0.8483, -0.7442],
        [-1.1195, -0.3819, -0.7515,  ..., -0.5175, -1.4563, -2.4408]])


In [8]:
# Adjacency matrix for the edges
print(data.adj_t)

SparseTensor(row=tensor([      0,       0,       0,  ..., 8388607, 8388607, 8388607]),
             col=tensor([      0,       1,       2,  ..., 1320822, 6800756, 8065975]),
             size=(8388608, 8388608), nnz=517557419, density=0.00%)


In [9]:
# Ground-truth labels
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([8388608])
tensor([36, 21, 11,  ...,  6, 46, 53])


In [10]:
# Train mask
print(f'train_mask = {data.train_mask.shape}')
print(data.train_mask)

train_mask = torch.Size([8388608])
tensor([False,  True,  True,  ...,  True,  True, False])


In [11]:
from torch_geometric.loader import NeighborLoader

# Create batches with neighbor sampling
train_loader = NeighborLoader(
    data,
    num_neighbors=[30],
    batch_size=2048,
    input_nodes=data.train_mask,
)

In [12]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(64, 64)
        self.optimizer = torch.optim.Adam(self.parameters(),
                                          lr=0.01,
                                          weight_decay=5e-4)

    def forward(self, x, adj_t):
        x = self.gcn1(x, adj_t)
        z = F.log_softmax(x, dim=1)
        return x, z

In [13]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data, train_loader, device):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 200
    total_elapsed_time = 0
    
    model.train()
    for epoch in range(epochs+1):
        # Training
        total_loss = 0
        acc = 0
        val_loss = 0
        val_acc = 0
        
        # Train on batches
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            h, out = model(batch.x, batch.adj_t)
            loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
            total_loss += loss
            acc += accuracy(out[batch.train_mask].argmax(dim=1), batch.y[batch.train_mask])
            loss.backward()
            optimizer.step()
                
        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {total_loss/len(train_loader):.3f} | Train Acc: '
                  f'{acc/len(train_loader)*100:>6.2f}%')
          
    return model, h, out

In [14]:
import time

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train and test
# Train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

start_time = time.process_time()
gcn_model, gcn_output, final_output = train(gcn.to(device), data.to(device), train_loader, device)
end_time = time.process_time()

elapsed_time = end_time - start_time

print()
print('Elapsed Time (CPU): ', elapsed_time, 'seconds')

GCN(
  (gcn1): GCNConv(64, 64)
)

Epoch   0 | Train Loss: 4.155 | Train Acc:   2.91%
Epoch  10 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch  20 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch  30 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch  40 | Train Loss: 4.152 | Train Acc:   2.91%
Epoch  50 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch  60 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch  70 | Train Loss: 4.152 | Train Acc:   2.91%
Epoch  80 | Train Loss: 4.152 | Train Acc:   2.91%
Epoch  90 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch 100 | Train Loss: 4.152 | Train Acc:   2.91%
Epoch 110 | Train Loss: 4.152 | Train Acc:   2.91%
Epoch 120 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch 130 | Train Loss: 4.152 | Train Acc:   2.91%
Epoch 140 | Train Loss: 4.152 | Train Acc:   2.92%
Epoch 150 | Train Loss: 4.152 | Train Acc:   2.93%
Epoch 160 | Train Loss: 4.152 | Train Acc:   2.93%
Epoch 170 | Train Loss: 4.152 | Train Acc:   2.90%
Epoch 180 | Train Loss: 4.152 | Train Acc:   2.9

In [ ]:
print(device)